# Assignment-02, Probability Model A First Look: An Introduction of Language Model

## Assignment

1. Review the course online programming code; 
2. Review the main questions; 
3. Using wikipedia corpus to build a language model. 

## 1. Review the course online programming code. 

*In this part, you should re-code the programming task in our online course.*

> 正则表达式
    > 1. 读语料库
    > 2. 爬虫

In [37]:
import pandas as pd
course_corpus = pd.read_csv('sqlResult_1558435.csv', encoding='gb18030')
all_articles = course_corpus.content.tolist()

In [38]:
import re
def token(string):
    return ' '.join(re.findall('[\w|\d]+', string))

TEXT = ''.join([token(str(string)) for string in all_articles])

In [39]:
print('length of TEXT is: {}'.format(len(TEXT)))

length of TEXT is: 37412506


In [40]:
# or use reduce to combine
from functools import reduce
txt_from_reduce = reduce(lambda a1, a2: token(str(a1)) + token(str(a2)), all_articles[:10])
print('length of txt_from_reduce is: {}'.format(len(txt_from_reduce)))
assert(TEXT[:len(txt_from_reduce)]==txt_from_reduce)

length of txt_from_reduce is: 12164


> 分词，获取语料库中所有词语


In [41]:
import jieba
def cut(string):
    return list(jieba.cut(string))

ALL_TOKENS = cut(TEXT)
VALID_TOKENS = [token for token in ALL_TOKENS if token.strip() and token!='n'] 

In [42]:
len(VALID_TOKENS)

17221785

> 计算词频

In [68]:
from collections import Counter
word_counts = Counter(VALID_TOKENS)

frequence_all = [f for w,f in word_counts.most_common()]
frequence_sum = sum(frequence_all)

In [33]:
def get_prob(word):
    esp = 1 / frequence_sum
    if word in word_counts:
        return word_counts[word] / frequence_sum
    else:
        return esp
    

In [34]:
def product(numbers):
    return reduce(lambda n1, n2: n1 * n2, numbers)


> one gram language model

In [35]:
def language_model_one_gram(string):
    words = cut(string)
    return product([get_prob(word) for word in words])

In [44]:
sentences = """
小明毕业于清华大学
小明毕业于秦华大学
""".split()

for s in sentences:
    print(s, language_model_one_gram(s))
    
# 句子越长，出现频率越低
# 未考虑顺序

need_compared = [
    "我去吃火锅，今晚 今晚我去吃火锅",
    "小明今天吃了一架飞机 小明今天吃了一架喷气式飞机",
    "小明今天吃了一个苹果 小明今天吃了一架喷气式飞机"
]

def sentence_compare(pairs, sentence_prob_func=language_model_one_gram):
    for s in pairs:
        s1, s2 = s.split()
        p1, p2 = sentence_prob_func(s1), sentence_prob_func(s2)
    
        better = s1 if p1 > p2 else s2
    
        print('{} is more possible'.format(better))
        print('-'*4 + ' {} with probility {}'.format(s1, p1))
        print('-'*4 + ' {} with probility {}'.format(s2, p2))
        
sentence_compare(need_compared)

小明毕业于清华大学 3.039960976514903e-18
小明毕业于秦华大学 5.813337476916057e-24
今晚我去吃火锅 is more possible
---- 我去吃火锅，今晚 with probility 6.876097222574346e-26
---- 今晚我去吃火锅 with probility 1.1841866800627252e-18
小明今天吃了一架飞机 is more possible
---- 小明今天吃了一架飞机 with probility 1.1408998714380734e-24
---- 小明今天吃了一架喷气式飞机 with probility 4.5710761764374054e-30
小明今天吃了一个苹果 is more possible
---- 小明今天吃了一个苹果 with probility 1.3072549452850427e-23
---- 小明今天吃了一架喷气式飞机 with probility 4.5710761764374054e-30


> two gram language model

In [65]:
all_2_gram_words = [''.join(VALID_TOKENS[i:i+2]) for i in range(len(VALID_TOKENS)-2)]
_2_gram_sum = len(all_2_gram_words)
_2_gram_counter = Counter(all_2_gram_words)
def get_combination_prob(word1, word2):
    if word1 + word2 in _2_gram_counter:
        return _2_gram_counter[word1 + word2] / _2_gram_sum
    else:
        return 1/_2_gram_sum

def get_prob_2_gram(word1, word2):
    return get_combination_prob(word1, word2) / get_prob(word1)
    

In [67]:
def language_model_of_2_gram(sentence):
    words = cut(sentence)
    sentence_prob = get_prob(words[0])
    for i in range(len(words)-2):
        sentence_prob *= get_prob_2_gram(words[i], words[i+1])
    
    return sentence_prob

print(language_model_of_2_gram('小明今天吃了一架飞机'))
print(language_model_of_2_gram('小明今天吃了一架喷气式飞机'))
print(language_model_of_2_gram('小明今天抽奖抽到一台苹果手机'))
print(language_model_of_2_gram('小明今天今天抽奖抽到一台波音飞机'))

3.783269766877993e-16
5.4435542535779205e-19
1.1122954608247838e-16
4.401644598329534e-20


In [69]:
need_compared = [
    "我去吃火锅，今晚 今晚我去吃火锅",
    "洋葱奶昔来一杯 养乐多绿来一杯",
    "小明今天吃了一架飞机 小明今天吃了一个苹果"
]

for s in need_compared:
    s1, s2 = s.split()
    p1, p2 = language_model_of_2_gram(s1), language_model_of_2_gram(s2)
    
    better = s1 if p1 > p2 else s2
    
    print('{} is more possible'.format(better))
    print('-'*4 + ' {} with probility {}'.format(s1, p1))
    print('-'*4 + ' {} with probility {}'.format(s2, p2))

今晚我去吃火锅 is more possible
---- 我去吃火锅，今晚 with probility 1.0580831504965566e-10
---- 今晚我去吃火锅 with probility 4.974106155814898e-10
养乐多绿来一杯 is more possible
---- 洋葱奶昔来一杯 with probility 1.1613200748517083e-08
---- 养乐多绿来一杯 with probility 5.8065996999265405e-08
小明今天吃了一个苹果 is more possible
---- 小明今天吃了一架飞机 with probility 3.783269766877993e-16
---- 小明今天吃了一个苹果 with probility 2.3624417877615912e-14


## 2. Review the main points of this lesson. 

##### 1. How to Github and Why do we use Jupyter and Pycharm; 

Ans: 

**github:**
+ 复制远程仓库到本地：git clone 远程仓库地址
+ 创建本地仓库：git init
+ 创建远程仓库：在github页面新建
+ 本地仓库连接到远程：git remote add origin 远程仓库地址
+ 提交代码：git add，git commit，git push
+ 拉取最新代码：git pull

**Why do we use Jupyter and Pycharm:**

Jupyter可以即时看到每行代码的效果，方便进行快速的实验，而且其中可以嵌入markdown，在代码中穿插讲解和图示、公式都很方便，适合作为向其他人展示结果或想法的载体；
Pycharm中有调试和代码分析的工具，而且可以清晰的展示项目结构，适合开发比较大的项目。

##### 2. What's the Probability Model?

Ans:概率模型是对随机事件建模，研究事件发生的概率。

##### 3. Can you came up with some sceneraies at which we could use Probability Model?

Ans:赌博中胜率的计算，或朴素贝叶斯模型，也是基于概率的，常用于垃圾邮件分类

##### 4. Why do we use probability and what's the difficult points for programming based on parsing and pattern match? 


Ans:

概率可以描述对一类事件的概括，可以帮助我们跳出对单个事件的分析（或减少分析的复杂性），帮助我们理解整体的规律。

基于解析和模式匹配的编程，需要人工从任务中总结出规律和规则，这个总结的过程本身就很复杂和繁琐，而用规则列举出所有可能的情况，也是很困难的，最后，用代码实现这些规则，也是一个复杂的过程，很容易出错。

##### 5. What's the Language Model;

Ans: 对一个句子成立的可能性建模。

#####  6. Can you came up with some sceneraies at which we could use Language Model?

Ans: 语音识别、机器翻译。

##### 7. What's the 1-gram language model;

Ans:句子成立的概率等于句子中所有词同时出现的概率，1-gram认为句子的中每个词出现的概率不受其他词的影响。

##### 8. What's the disadvantages and advantages of 1-gram language model;

Ans:1-gram的假设比较强，优点是简化了模型，容易实现，缺点是出错的可能性也更大，因为现实中存在很多与假设不相容的情况。

##### 9.  What't the 2-gram models; 

Ans:句子成立的概率等于句子中所有词同时出现的概率，2-gram认为句子的中每个词出现的概率仅受他前面1个词的影响。

##### 10. what's the web crawler, and can you implement a simple crawler? 

Ans:爬虫就是从网页或应用爬取数据的工具。
> 实现:豆瓣图书榜单TOP250爬取

In [110]:
import requests
from bs4 import BeautifulSoup


In [275]:
# 豆瓣读书榜单TOP250爬取
import requests
url = 'https://book.douban.com/top250?start='
tag_p = re.compile('<.*?>')
books = []

for page in range(10):  # 每页25个
    url_content = requests.get(url+str(page))
    soup = BeautifulSoup(url_content.text)
    book_infos = [tr.select('div')[:2] for tr in soup.select('div #content')[0].find_all('tr')]
    for book_raw in book_infos:
        title = ' '.join([tag_p.sub('', str(t)).strip() for t in book_raw[0].select('a')[0].contents])
    
        rate = book_raw[1].select('span')[1].string.strip()
        rate_people = re.search('\d+', book_raw[1].select('span')[2].string.strip()).group()
        books.append((title, rate, rate_people))
    
    

In [277]:
books  #(标题，评分，评分人数)

[('追风筝的人', '8.9', '429859'),
 ('解忧杂货店', '8.5', '415928'),
 ('小王子', '9.0', '352285'),
 ('白夜行', '9.1', '307373'),
 ('围城', '8.9', '258855'),
 ('三体 : “地球往事”三部曲之一 ', '8.8', '254334'),
 ('嫌疑人X的献身', '8.9', '251389'),
 ('活着', '9.3', '211133'),
 ('挪威的森林', '8.0', '243039'),
 ('百年孤独', '9.2', '188639'),
 ('红楼梦', '9.6', '179288'),
 ('看见', '8.8', '183508'),
 ('平凡的世界（全三部）', '9.0', '155592'),
 ('三体Ⅱ : 黑暗森林 ', '9.3', '150060'),
 ('不能承受的生命之轻', '8.5', '165204'),
 ('三体Ⅲ : 死神永生 ', '9.2', '148169'),
 ('达·芬奇密码', '8.2', '162640'),
 ('我们仨', '8.7', '140479'),
 ('天才在左 疯子在右 : 国内第一本精神病人访谈手记 ', '8.1', '148488'),
 ('简爱（英文全本）', '8.5', '137407'),
 ('哈利·波特与魔法石', '9.0', '121909'),
 ('明朝那些事儿（壹） : 洪武大帝 ', '8.7', '123393'),
 ('傲慢与偏见', '8.8', '122811'),
 ('目送', '8.7', '123586'),
 ('恶意', '8.4', '119000'),
 ('解忧杂货店', '8.5', '415928'),
 ('小王子', '9.0', '352285'),
 ('白夜行', '9.1', '307373'),
 ('围城', '8.9', '258855'),
 ('三体 : “地球往事”三部曲之一 ', '8.8', '254334'),
 ('嫌疑人X的献身', '8.9', '251389'),
 ('活着', '9.3', '211133'),
 ('挪威的森林', '8.0'

##### 11.  There may be some issues to make our crwaler programming difficult, what are these, and how do we solve them?

Ans:网站可能会有很多反爬机制，如禁止某些cookie或agent或不带cookie的请求，封锁过于频繁请求的ip，或者存在很多页面跳转；解决方法：在爬虫中带上真实浏览器的cookie和agent，用多个ip轮流访问，或者用selenium模拟真实用户的操作。

##### 12. What't the Regular Expression and how to use?

Ans:正则表达式描述了一个字符串的构成或者模式，可以用来做字符串的模式匹配。

## 3. Using Wikipedia dataset to finish the language model. 

Step 1: You need to download the corpus from wikipedis:
> https://dumps.wikimedia.org/zhwiki/20190401/

Step 2: You may need the help of wiki-extractor:

> https://github.com/attardi/wikiextractor

In [2]:
!../wikiextractor/WikiExtractor.py -o wiki_expand_corpus -b 250K -q ../../../data/zhwiki-20190401-pages-articles.xml

../wikiextractor/WikiExtractor.py:2432: DeprecationWarning: Flags not at the start of the expression '\\[(((?i)bitcoin:|ftp' (truncated)
  re.S | re.U)
../wikiextractor/WikiExtractor.py:2439: DeprecationWarning: Flags not at the start of the expression '^(http://|https://)(' (truncated)
  re.X | re.S | re.U)
../wikiextractor/WikiExtractor.py:645: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  self.title, self.id, *errs)
../wikiextractor/WikiExtractor.py:645: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  self.title, self.id, *errs)
../wikiextractor/WikiExtractor.py:645: DeprecationWarning: The 'warn' function is deprecated, use 'warning' instead
  self.title, self.id, *errs)


In [6]:
!du -h wiki_expand_corpus

 24M	wiki_expand_corpus/AX
 24M	wiki_expand_corpus/BW
 24M	wiki_expand_corpus/BP
 24M	wiki_expand_corpus/BY
 24M	wiki_expand_corpus/AV
 24M	wiki_expand_corpus/AQ
 24M	wiki_expand_corpus/BL
 24M	wiki_expand_corpus/AC
 24M	wiki_expand_corpus/BK
 24M	wiki_expand_corpus/AD
 24M	wiki_expand_corpus/AM
 24M	wiki_expand_corpus/BB
 24M	wiki_expand_corpus/AJ
 24M	wiki_expand_corpus/BE
 24M	wiki_expand_corpus/AP
 24M	wiki_expand_corpus/BX
 24M	wiki_expand_corpus/AW
 24M	wiki_expand_corpus/BQ
 24M	wiki_expand_corpus/AY
 24M	wiki_expand_corpus/BV
 24M	wiki_expand_corpus/AK
 24M	wiki_expand_corpus/BD
 24M	wiki_expand_corpus/AL
 24M	wiki_expand_corpus/BC
 24M	wiki_expand_corpus/BJ
 24M	wiki_expand_corpus/AE
 24M	wiki_expand_corpus/BM
 23M	wiki_expand_corpus/AB
 15M	wiki_expand_corpus/CA
 25M	wiki_expand_corpus/BH
 24M	wiki_expand_corpus/AG
 24M	wiki_expand_corpus/BO
 24M	wiki_expand_corpus/AI
 24M	wiki_expand_corpus/BF
 24M	wiki_expand_corpus/AN
 24M	wiki_expand_corpus/BA
 24M	wiki_expand_corpus/BS
 

In [12]:
!ls -Aq wiki_expand_corpus | wc -l

      53


Step 3: Using the technologies and methods to finish the language model; 
> 


In [1]:
# 1. read file
import os
wiki_articles = []
for sub_folder in os.listdir('wiki_expand_corpus/'):
    if sub_folder[0]!='.':
        for fname in os.listdir('wiki_expand_corpus/'+sub_folder)[:10]:
            filepath = 'wiki_expand_corpus/'+sub_folder+'/'+fname
#             print(filepath)
            wiki_articles.append(open(filepath).read())


In [3]:
import re
tag_pattern = re.compile('<.*?>')
def token_wiki(string, del_pattern):
    
    return ' '.join(re.findall('[\w|\d]+', del_pattern.sub('', string)))


In [4]:
token_wiki(wiki_articles[0], tag_pattern)

'李元曉 李元晓 唐高祖李淵第二十一子 母魯才人 生年不详 从异母兄李元祥生年推断 当在626年或之后 亦曾有傳其為628年出生 631年 封密王 635年 任虢州刺史 640年 受实封八百户 649年 加增实封千户 转任泽州刺史 653年 任宣州刺史 後转任徐州刺史 676年去世 追贈司徒 揚州都督 陪葬献陵 谥号貞 子南安王李穎袭位 另一子李亮 李元祥 李元祥 唐高祖李淵第二十子 母为杨素之女楊嬪 601年 657年 631年 封許王 637年 徙封江王 任蘇州刺史 受实封八百户 649年 加增实封千户 唐高宗即位后 历任金州 鄜州 鄭州刺史 李元祥性格贪婪 好搜集金宝 贪得无厌 与弟弟滕王李元嬰 侄儿蒋王李惲 哥哥虢王李鳳以貪欲暴虐闻名 王府官属都不愿做他们的下属 说 宁向儋 崖 振 白 宁可流放到儋州 今海南儋县 崖州 今海南琼山 振州 今海南三亚 白州 今广西博白 不事江 滕 蒋 虢 元祥体态宽大 腰帯相当于十个人的腰围 饭量相当于数人 当時韓王李元嘉 虢王李鳳 魏王李泰都体态宽大 但都不及李元祥 李元祥还瞎了一个眼 据其墓志记载 李元祥逝世于调露二年七月廿七日 680年9月7日 春秋五十有五 追贈司徒 并州大都督 陪葬献陵 谥号安 7子 后来李元祥孙武阳郡王李堪继嗣为澧国公 其为何人之子不详 房陵公主 房陵公主 唐高祖李淵第六女 621年11月 封永嘉公主 下嫁竇奉節 生下女儿竇氏 654年 改封為房陵大長公主 後來又下嫁賀蘭僧伽 房陵公主先後嫁了兩個丈夫 她的前夫名竇奉節 是高祖李淵竇皇后的侄兒 不但出身高貴 且擔任左衛將軍 秦州都督等職務 但是永嘉公主並不把丈夫放在眼裡 一邊監視丈夫 不許拈花惹草 另一邊包養了不少情夫 永嘉公主的情夫裡 有一個人物名叫楊豫之 是唐太宗李世民胞弟巢刺王李元吉之女壽春縣主的丈夫 永嘉公主和壽春縣主的年紀雖然相彷 但永嘉公主卻是壽春縣主的姑姑 不過她卻和侄女的丈夫幽會 而楊豫之也無所顧忌 他宣稱自己死了母親 要做孝子 不忍有房閨之樂 因此要跟老婆分居 趁機躲開壽春縣主的監視 和永嘉公主通姦 楊豫之很快就被竇奉節帶兵殺去 但竇奉節不久後也死了 丈夫和情夫先後死去 永嘉公主嫁给賀蘭僧伽 而身邊的情夫更猶如雜草般不斷地冒出来 賀蘭僧伽對永嘉公主的淫亂不聞不問 賀蘭僧伽在咸亨四年673年死去 永嘉公主於閏五月初三 在九成宮避暑後去

In [5]:
WIKI_TEXT = ' '.join([token_wiki(article, tag_pattern) for article in wiki_articles])

In [7]:
print('length of WIKI_TEXT is:{}'.format(WIKI_TEXT))
print('first 100 tokens:'+WIKI_TEXT[:100])

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
# 2. get token (one word and two words)
ALL_WIKI_TOKENS = cut(WIKI_TEXT) 
VALID_WIKI_TOKENS = [t for t in ALL_WIKI_TOKENS if t.strip()]
# 3. build language model

In [236]:
from collections import Counter
wiki_counter = Counter(VALID_WIKI_TOKENS)
wiki_counter.most_common(100)

[('的', 1011101),
 ('年', 308915),
 ('在', 288196),
 ('是', 228795),
 ('和', 150958),
 ('月', 136832),
 ('了', 136769),
 ('於', 130712),
 ('為', 96037),
 ('有', 87285),
 ('日', 79657),
 ('他', 77974),
 ('被', 75641),
 ('中', 74761),
 ('为', 71967),
 ('人', 71477),
 ('也', 65909),
 ('與', 63968),
 ('後', 59509),
 ('由', 57155),
 ('而', 57027),
 ('1', 50592),
 ('上', 48804),
 ('等', 48396),
 ('以', 43577),
 ('2', 41098),
 ('但', 39178),
 ('3', 39003),
 ('及', 38395),
 ('到', 37616),
 ('一个', 35977),
 ('与', 35490),
 ('于', 33782),
 ('5', 33462),
 ('4', 32326),
 ('都', 32178),
 ('其', 32130),
 ('至', 30289),
 ('6', 29203),
 ('之', 28850),
 ('或', 28104),
 ('對', 27305),
 ('可以', 27220),
 ('后', 26711),
 ('10', 26489),
 ('7', 26473),
 ('不', 26140),
 ('8', 26117),
 ('中国', 25456),
 ('日本', 24759),
 ('并', 24590),
 ('人口', 24279),
 ('12', 23826),
 ('9', 23442),
 ('使用', 22656),
 ('來', 22601),
 ('以及', 22248),
 ('香港', 22178),
 ('其中', 21199),
 ('就', 21065),
 ('一個', 20866),
 ('她', 20606),
 ('米', 20338),
 ('曾', 20235),
 ('平方公里', 20217),
 

In [235]:
# 3. build language model
class BaseFreqEstimator():
    def fit(self, baseCounts):
        self.baseCounts = baseCounts
        self.freq_sum = sum([f for w,f in baseCounts.most_common()])
        self.eps = 1 / self.freq_sum
        
    def get_smoothed_probability(self, word):
        if word in self.baseCounts:
            return self.baseCounts[word] / self.freq_sum
        else:
            return self.eps

class LanguageModelTwoGram():
    def __init__(self, tokens, corpus=None, token_generator=None, freq_estimator_name=BaseFreqEstimator.__name__):
        assert((tokens is not None) or (corpus is not None))
    
        if tokens is None:
            if token_generator is None:
                all_tokens = cut(corpus) 
                self.tokens = [t for t in all_tokens if t.strip()]
            else:
                self.tokens = token_generator(corpus)
        else:
            self.tokens = tokens
            
        self.freq_estimator_1g, self.freq_estimator_2g = (globals()[freq_estimator_name](),globals()[freq_estimator_name]())
       
    @staticmethod
    def cut(sentenct):
        return list(jieba.cut(sentence))

    def fit(self):
        self.word_counter = Counter(self.tokens)
        self.freq_estimator_1g.fit(self.word_counter)
#         self.freq_sum = sum(self.word_counter.values())
#         self.eps = 1 / self.freq_sum
        
        two_words = [''.join(self.tokens[i:i+2]) for i in range(len(self.tokens)-2)]
        self.two_word_counter = Counter(two_words)
        self.freq_estimator_2g.fit(self.two_word_counter)
#         self.two_word_freq_sum = len(two_words)
#         self.two_word_eps = 1 / self.two_word_freq_sum
        
#     def get_prob(self, word):
#         if word in self.word_counter:
#             return self.word_counter[word] / self.freq_sum
#         else:
#             return self.eps
        
#     def get_combination_prob(self, word1, word2):
#         if word1 + word2 in self.two_word_counter:
#             return self.two_word_counter[word1 + word2] / self.two_word_freq_sum
#         else:
#             return self.two_word_eps
    
    def get_2_gram_prob(self, word1, word2):
        return self.freq_estimator_2g.get_smoothed_probability(word1 + word2) / self.freq_estimator_1g.get_smoothed_probability(word1)
    
    def sentence_prob(self, sentence):
        words = cut(sentence)
        
        prob = self.freq_estimator_1g.get_smoothed_probability(words[0])
        for i in range(len(words)-1):
            prob *= self.get_2_gram_prob(words[i], words[i+1])
            
        return prob
        

Step 4: Try some interested sentence pairs, and check if your model could fit them

>

In [86]:
samples = """
小明毕业于清华大学
小明毕业于秦华大学
小明今天吃了一架飞机
小明今天吃了一架喷气式飞机
回酋投荒窜死荒岛，雄狮追寇穷极遐方
这是怎么回事？
为什么我内心好像有一种可鄙可耻的感觉？
是不是因为我要去干流血的勾当？
怎么肥四？
希望各大出版社编辑、设计都能以在书的封面、扉页剧透为耻，切切！
""".split()

need_compared = [
    "小明今天抽奖抽到一台波音飞机 小明今天抽奖抽到一台苹果手机",
    "洋葱奶昔来一杯 养乐多绿来一杯",
    "小明今天吃了一架飞机 小明今天吃了一个苹果",
    "卡尔卡西传统理念与塔雷加现代指法的完美结合 卡尔卡西传统老茧与塔雷加现代水泡的完美结合",
    "今天，让我们来做一份美味的洁厕灵 今天，让我们来做一份美味的Tiramisu",
    "首先，用滚水冲开HCL 首先，用滚水冲开Expresso粉",
    "这是一个复杂的优化问题 这是一个不能说不复杂的优化问题"
]



In [103]:
lm_2g = LanguageModelTwoGram(VALID_WIKI_TOKENS)
lm_2g.fit()

In [96]:
words = cut("首先，用滚水冲开Expresso粉")
for word in words:
    print(word,lm_2g.word_counter[word])
for i in range(len(words)-1):
    print(words[i], words[i+1], lm_2g.two_word_counter[words[i]+words[i+1]])

首先 1508
， 0
用 10389
滚水 0
冲开 2
Expresso 0
粉 259
首先 ， 0
， 用 0
用 滚水 0
滚水 冲开 0
冲开 Expresso 0
Expresso 粉 0


In [126]:
for sen in samples:
    print(sen, lm_2g.sentence_prob(sen))
sentence_compare(need_compared, lm_2g.sentence_prob)

小明毕业于清华大学 1.6088501168378184e-11
小明毕业于秦华大学 6.995001185650172e-13
小明今天吃了一架飞机 1.358784116421255e-17
小明今天吃了一架喷气式飞机 6.176292036623426e-19
回酋投荒窜死荒岛，雄狮追寇穷极遐方 5.4705706575754825e-16
这是怎么回事？ 2.42194307552183e-08
为什么我内心好像有一种可鄙可耻的感觉？ 2.94062164129477e-28
是不是因为我要去干流血的勾当？ 1.3179389622669033e-27
怎么肥四？ 4.84388615104366e-08
希望各大出版社编辑、设计都能以在书的封面、扉页剧透为耻，切切！ 6.285931536602777e-39
小明今天抽奖抽到一台苹果手机 is more possible
---- 小明今天抽奖抽到一台波音飞机 with probility 1.6649128233530405e-17
---- 小明今天抽奖抽到一台苹果手机 with probility 7.357633977048629e-17
养乐多绿来一杯 is more possible
---- 洋葱奶昔来一杯 with probility 7.573306160582044e-12
---- 养乐多绿来一杯 with probility 4.84388615104366e-08
小明今天吃了一架飞机 is more possible
---- 小明今天吃了一架飞机 with probility 1.358784116421255e-17
---- 小明今天吃了一个苹果 with probility 1.6003113261528598e-18
卡尔卡西传统老茧与塔雷加现代水泡的完美结合 is more possible
---- 卡尔卡西传统理念与塔雷加现代指法的完美结合 with probility 2.4971089346974124e-34
---- 卡尔卡西传统老茧与塔雷加现代水泡的完美结合 with probility 8.379333788413579e-32
今天，让我们来做一份美味的Tiramisu is more possible
---- 今天，让我们来做一份美味的洁厕灵 

 

Step 5: If we need to solve following problems, how can language model help us? 

+ Voice Recognization.
+ Sogou *pinyin* input.
  + 前两个问题，可以看成是有输入一组符号或音节，需对应到一组词的问题，这组词假设有很多种可能（路径），语言模型可以用来作为一个指标，即输出概率最大的那一条路径
+ Auto correction in search engine. 
  + 将所有用户搜索过的词按照概率（语言模型）排序，假设输入搜索词A，从上往下找包含词A的搜索词
+ Abnormal Detection.
  + 把事件看作词语，如果某一事件序列的概率（语言模型）比较低，那么这个事件序列可能是异常的，或者包含异常事件

### Compared to the previous learned parsing and pattern match problems. What's the advantage and disavantage of Probability Based Methods? 

Ans: 优点是代码实现简单，而且代码完成后，可以重复使用，如果想要识别新的模式，只要增加数据即可，不需要设计模式；缺点是需要大量的数据来训练，例如上述使用部分wiki数据训练的模型，对于比较口语化的句子，或是包含没见过的词语的句子，计算的概率就会不准确

## (Optional)  How to solve *OOV* problem?

If some words are not in our dictionary or corpus. When we using language model, we need to overcome this `out-of-vocabulary`(OOV) problems. There are so many intelligent man to solve this probelm. 

-- 

The first question is: 

**Q1: How did you solve this problem in your programming task?**

Ans: 令他们的概率等于一个比较小的常数：1/语料库大小

Then, the sencond question is: 

**Q2: Read about the 'Turing-Good Estimator', can explain the main points about this method, and may implement this method in your programming task**

Reference: 
+ https://www.wikiwand.com/en/Good%E2%80%93Turing_frequency_estimation
+ https://github.com/Computing-Intelligence/References/blob/master/NLP/Natural-Language-Processing.pdf, Page-47

> coding in here

**Good-Turing Estimator**
基本思想是把出现次数多的种类（词）的概率分给未出现过的种类，但出现越多的种类反而分的越少。
1. 如果某词未出现过，则对他估计的概率为
$$p_0=\frac{N_1}{N}$$
2. 如果出现过r次，则估计概率为
$$p_r = \frac{(r+1)E(N_{r+1})}{NE(N_r)}$$
这里用$p_r=\frac{r^*}{N}$估计$p_r$，然后根据基本思想，令$r^*=\frac{(r+1)E(N_{r+1})}{E(N_r)}$（这里为什么，其实还是不太懂）。
3. 对于比较小的r，$E(N_r)$可以认为就等于$N_r$，r比较大时，需要用$S(N_r)$来估计。
Church和Gale提出的计算$S(N_r)$的方法（smoothing method）：
  - a.用$Z_r$估计$N_r$，r越大，Nr等于0的越多（越稀疏），t-q就越大，$Z_r$对$N_r$修正也越多：
$$Z_r=\frac{N_r}{0.5(t-q)}$$
t,q是r前后离他最近的两个非0频率的频率的角标。
  - b.用线性回归拟合函数，然后从直线上取$S(N_r)$的值：$S(N_r)=Z_r$
$$log(Z_r)=a+blog(r)$$
如果拟合结果b不满足：b<-1，则smooth方法不可用。
4.未出现的词概率估计：
直接用$p_0$存在未出现词的概率可能高于出现过词的概率的情况，且直接用$p_0$，所有词的概率和其实大于1，导致在2gram模型中，w1w2的概率高于w1的概率（假如w1w2未出现过），于是$p(w1w2|w1)>1$。
因此，以下代码使用了老师给的解决办法：
  用出现次数为1的词中，长度与未出现的词w长度相等的词的个数来估计p(w);如果没有长度相等的词，就在$p_0$的基础上除以一个与词长度相关的衰减系数。

In [297]:
from sklearn import linear_model
import numpy as np
class GoodTuringEstimator():
    def __init__(self, N_threshold=5, prior=1e-8, p0_base=1/26.):
        self.N_threshold = N_threshold
        self.prior = prior
        self.p0_base = p0_base
        
    def fit(self, base_counts):
        self.base_counts = base_counts
        freq = [f for w,f in base_counts.most_common()]
        self.total_count = sum(freq)
        
        self.ones = Counter([len(w) for w,f in base_counts.most_common()])
        self.ones_longest = max(self.ones)
        
        self.freq_of_freq = Counter(freq)
        assert(self.total_count == sum([f*ff for f,ff in self.freq_of_freq.most_common()]))
        self.Nr_regression = linear_model.LinearRegression()
        
        self.fit_line()
        
    def fit_line(self):
        """
        拟合直线log(Z_r)=a+blog(r)
        """
        freq_counts = list(self.freq_of_freq.items())
        freq_counts.sort(key=lambda x:x[0])
        pad_freq = [0] + [f[0] for f in freq_counts] + [2*freq_counts[-1][0]-freq_counts[-2][0]]
        Zr = [2*freq_counts[i][1]/(pad_freq[i+2]-pad_freq[i]) for i in range(len(freq_counts))]
        self.Nr_regression.fit(np.log(pad_freq[1:-1]).reshape(-1,1), np.log(Zr))
        if self.Nr_regression.coef_[0]>=-1:
            raise Exception("Failure: linear regression model's slope is bigger than -1")
    

    def get_smoothed_freq_of_freq(self, times):
        if times<self.N_threshold:
            return self.freq_of_freq[times]
        else:
            return np.exp(self.Nr_regression.predict([[np.log(times)]])[0])

    def get_smoothed_probability(self, species):
        naive_count = self.base_counts[species]
        if naive_count == 0:
            if len(species) in self.ones:
                return self.prior * self.ones[len(species)] / self.total_count
            else:
                return self.prior * self.freq_of_freq[1] / self.total_count * self.p0_base ** (len(species)-self.ones_longest)
        else:
            return (naive_count+1)*self.get_smoothed_freq_of_freq(naive_count+1) / \
                (self.total_count*self.get_smoothed_freq_of_freq(naive_count))

In [298]:
# lm_2g_sgt.freq_estimator_1g.get_smoothed_probability('我')
naive_count = lm_2g_sgt.freq_estimator_1g.base_counts['秦华']
lm_2g_sgt.freq_estimator_1g.get_smoothed_freq_of_freq(naive_count+1) #/ \
#                 (lm_2g_sgt.freq_estimator_1g.total_count*lm_2g_sgt.freq_estimator_1g.get_smoothed_freq_of_freq(naive_count))

1053811.247882889

In [280]:
lm_2g_sgt.freq_estimator_1g.base_counts['我']

9297

In [219]:
lm_2g_sgt.freq_estimator_1g.get_smoothed_probability('我')

0.00045029283811023267

In [287]:
base_counts = Counter(VALID_WIKI_TOKENS)

In [288]:
# N_1
sum([f==1 for w,f in base_counts.most_common()])

595723

In [289]:
# N
sum([f for w,f in base_counts.most_common()])

20644585

In [218]:
lm_2g_sgt.freq_estimator_1g.freq_of_freq[1] / lm_2g_sgt.freq_estimator_1g.total_count

0.02885613830454814

In [224]:
lm_2g_sgt.freq_estimator_1g.total_count

20644585

In [281]:
lm_2g_sgt.freq_estimator_1g.freq_of_freq[1]

595723

In [214]:
lm_2g_sgt.freq_estimator_1g.Nr_regression.coef_

array([-1.89131939])

In [211]:
np.exp(lm_2g_sgt.freq_estimator_1g.Nr_regression.predict([[np.log(9298)]])[0])

0.0329056689090447

In [168]:
freq, freq_freq = zip(*[(f,ff) for f,ff in lm_2g_sgt.freq_estimator_1g.freq_of_freq.most_common()])

In [202]:
freq_counts = list(lm_2g_sgt.freq_estimator_1g.freq_of_freq.items())
freq_counts.sort(key=lambda x:x[0])
pad_freq = [0] + [f[0] for f in freq_counts] + [2*freq_counts[-1][0]-freq_counts[-2][0]]
Zr = [2*freq_counts[i][1]/(pad_freq[i+2]-pad_freq[i]) for i in range(len(freq_counts))]

In [203]:
Zr

[595723.0,
 148717.0,
 68240.0,
 40465.0,
 26616.0,
 19715.0,
 14930.0,
 12037.0,
 9893.0,
 8123.0,
 6909.0,
 5991.0,
 5207.0,
 4655.0,
 4131.0,
 3735.0,
 3239.0,
 3099.0,
 2765.0,
 2470.0,
 2370.0,
 2175.0,
 1960.0,
 1861.0,
 1832.0,
 1668.0,
 1480.0,
 1474.0,
 1323.0,
 1295.0,
 1162.0,
 1152.0,
 1066.0,
 1017.0,
 951.0,
 950.0,
 888.0,
 779.0,
 804.0,
 789.0,
 774.0,
 737.0,
 724.0,
 678.0,
 633.0,
 603.0,
 644.0,
 647.0,
 598.0,
 500.0,
 516.0,
 523.0,
 479.0,
 462.0,
 485.0,
 497.0,
 412.0,
 441.0,
 423.0,
 377.0,
 379.0,
 345.0,
 370.0,
 344.0,
 340.0,
 308.0,
 309.0,
 317.0,
 331.0,
 282.0,
 302.0,
 288.0,
 288.0,
 259.0,
 247.0,
 270.0,
 242.0,
 258.0,
 246.0,
 245.0,
 212.0,
 230.0,
 240.0,
 248.0,
 217.0,
 189.0,
 202.0,
 203.0,
 179.0,
 189.0,
 163.0,
 167.0,
 186.0,
 194.0,
 156.0,
 158.0,
 167.0,
 134.0,
 181.0,
 151.0,
 159.0,
 136.0,
 155.0,
 121.0,
 129.0,
 152.0,
 142.0,
 130.0,
 132.0,
 145.0,
 132.0,
 139.0,
 129.0,
 128.0,
 118.0,
 131.0,
 115.0,
 133.0,
 123.0,
 121

In [199]:
freq_counts[-1]

(1011101, 1)

In [194]:
308915*2-288196

329634

In [200]:
pad_freq_counts[-2:]

[(1011101, 1), (1713287, -1)]

In [184]:
freq_counts[-5:-1]

[(150958, 1), (228795, 1), (288196, 1), (308915, 1)]

In [299]:
lm_2g_sgt = LanguageModelTwoGram(tokens=VALID_WIKI_TOKENS, freq_estimator_name=GoodTuringEstimator.__name__)
lm_2g_sgt.fit()

In [300]:
for sen in samples:
    print(sen, lm_2g_sgt.sentence_prob(sen))
sentence_compare(need_compared, lm_2g_sgt.sentence_prob)

小明毕业于清华大学 5.973362875979885e-13
小明毕业于秦华大学 5.966930198231084e-15
小明今天吃了一架飞机 1.1536593866980703e-19
小明今天吃了一架喷气式飞机 8.63023568326057e-23
回酋投荒窜死荒岛，雄狮追寇穷极遐方 6.107219560230259e-22
这是怎么回事？ 1.4103144220922233e-10
为什么我内心好像有一种可鄙可耻的感觉？ 1.0728254830912797e-34
是不是因为我要去干流血的勾当？ 3.6949028797951556e-35
怎么肥四？ 1.0585679772446538e-08
希望各大出版社编辑、设计都能以在书的封面、扉页剧透为耻，切切！ 5.945928398190028e-45
小明今天抽奖抽到一台苹果手机 is more possible
---- 小明今天抽奖抽到一台波音飞机 with probility 1.25989233377729e-25
---- 小明今天抽奖抽到一台苹果手机 with probility 9.670488371033475e-24
养乐多绿来一杯 is more possible
---- 洋葱奶昔来一杯 with probility 3.693161180765263e-14
---- 养乐多绿来一杯 with probility 7.505022130579898e-09
小明今天吃了一架飞机 is more possible
---- 小明今天吃了一架飞机 with probility 1.1536593866980703e-19
---- 小明今天吃了一个苹果 with probility 8.315674330231443e-21
卡尔卡西传统老茧与塔雷加现代水泡的完美结合 is more possible
---- 卡尔卡西传统理念与塔雷加现代指法的完美结合 with probility 5.384165867663883e-43
---- 卡尔卡西传统老茧与塔雷加现代水泡的完美结合 with probility 2.3368123421202087e-40
今天，让我们来做一份美味的Tiramisu is more possible
---- 今天，让我们来做一份美味的洁